In [2]:
import psycopg2
import pandas as pd
import numpy as np
import os

## Exploramos tabla históricos "HIS_Elemento_Posiciones" y La de los Elementos "B_Elementos"

In [4]:
# cargar toda la tabla B_Elementos
conexion = psycopg2.connect(
    host="srvbu-bd",
    port="5432",
    database="BITAL_SACYR_ICONICA",
    user="adminpg",
    password="KRZEy5fxQyNmmVjeUseQ",
)

cursor = conexion.cursor()

cursor.execute('SELECT * FROM public."B_Elementos" WHERE "Fo_Tipoelemento" = 5;')
# Obtener todos los registros
registros = cursor.fetchall()

# crear un df con pandas
columnas = [desc[0] for desc in cursor.description]
df_elementos = pd.DataFrame(registros, columns=columnas)

# Filtramos los nombres en ["M-108-0", "M-139-0", "M-43-0", "M-267-0", "CU-801", "M-109-0", "M-110-0"]
df_elementos = df_elementos[
    df_elementos["Nombre"].isin(["M-108-0", "M-139-0", "M-43-0", "M-267-0", "CU-801-0", "M-109-0", "M-110-0"])
]

# seleccionamos los pares Nombre, Id
for nombre, Id in zip(df_elementos["Nombre"], df_elementos["Id"]):
    print(f'"{nombre}": {Id},')


display(df_elementos)

"M-267-0": 16,
"M-109-0": 18,
"M-110-0": 20,
"CU-801-0": 23,
"M-108-0": 10,
"M-139-0": 11,
"M-43-0": 12,


,Id,Nombre,Fo_Tipoelemento,FechaBaja,TR_Coordenadas,TR_Ocupacion,TR_IntensidadCarrilAsc,TR_VelMedia,TR_Fecha,TR_Sentido_EsAsce,TR_Carriles,TR_IntensidadCarrilDesc,PuntoKm
4,16,M-267-0,5,None,0101000020E6100000CE531D72332A444040A374E95F32...,None,44.0,None,NaT,False,NaN,44.0,None
6,18,M-109-0,5,None,0101000020E61000003E3C4B901126444064744012F6CD...,None,66.0,None,NaT,False,NaN,66.0,None
8,20,M-110-0,5,None,0101000020E6100000AB984A3FE1204440AA0D4E44BFD6...,None,88.0,None,NaT,False,NaN,88.0,None
10,23,CU-801-0,5,None,0101000020E6100000AF0AD462F0024440EE7A698A0047...,None,1.0,None,NaT,False,NaN,1.0,None
15,10,M-108-0,5,None,0101000020E610000059501894693244404CFF9254A618...,None,300.0,None,2025-10-08 10:20:00,False,3.0,900.0,0.00
16,11,M-139-0,5,None,0101000020E610000087359545613144405DC2A1B778D8...,None,400.0,None,NaT,False,3.0,700.0,2.00
17,12,M-43-0,5,None,0101000020E610000034BF9A03042F44405A4B0169FFA3...,None,350.0,None,NaT,False,3.0,1200.0,5.00


In [ ]:
# Obtener todas las filas de HIS_Elemento_Posiciones para los Id de df_elementos

ids = df_elementos["Id"].dropna().astype(int).tolist()

if not ids:
    print("No hay Id en df_elementos.")
else:
    consulta = 'SELECT * FROM public."HIS_Elemento_Posiciones" WHERE "Fo_Elemento" = ANY(%s::int[]) ORDER BY "Fo_Elemento";'
    cursor.execute(consulta, (ids,))
    resultados = cursor.fetchall()
    columnas_pos = [desc[0] for desc in cursor.description]
    df_posiciones = pd.DataFrame(resultados, columns=columnas_pos)
    display(df_posiciones.head(5))


# contamos todas las filas de df que contienen cada Id
print(df_posiciones.value_counts("Fo_Elemento"))

# min max date per Id
print(df_posiciones.groupby("Fo_Elemento")["Fecha"].agg([np.min, np.max]))

# concadenamos la Id y la Fecha para crear un Id_compouesta unico
df_posiciones["Id_compouesta"] = df_posiciones["Fo_Elemento"].astype(str) + "_" + df_posiciones["Fecha"].astype(str)
print(df_posiciones.value_counts("Id_compouesta"))


,Id,Fo_Elemento,Coordenadas,Ocupacion,IntensidadCarrilAsc,VelMedia,Fecha,Sentido_EsAscendente,Carriles,IntensidadCarrilDesc
0,1032749,10,0101000020E610000059501894693244404CFF9254A618...,None,1967.00,None,2022-01-01 00:00:00,None,None,1074.00
1,1032750,10,0101000020E610000059501894693244404CFF9254A618...,None,1128.00,None,2022-01-01 01:00:00,None,None,1606.00
2,1032751,10,0101000020E610000059501894693244404CFF9254A618...,None,582.00,None,2022-01-01 02:00:00,None,None,916.00
3,1032752,10,0101000020E610000059501894693244404CFF9254A618...,None,302.00,None,2022-01-01 03:00:00,None,None,433.00
4,1032753,10,0101000020E610000059501894693244404CFF9254A618...,None,282.00,None,2022-01-01 04:00:00,None,None,274.00


Fo_Elemento
12    26066
10    25680
16    25304
18    17019
11    15961
20    11567
Name: count, dtype: int64
                   min                 max
Fo_Elemento                               
10          2022-01-01 2024-12-31 23:00:00
11          2022-01-01 2023-10-31 23:00:00
12          2022-01-01 2024-12-31 23:00:00
16          2022-01-01 2024-11-27 23:00:00
18          2022-01-01 2024-12-31 23:00:00
20          2022-01-01 2023-06-20 23:00:00


C:\Users\inaki.campo\AppData\Local\Temp\ipykernel_9160\3731279526.py:20: FutureWarning: The provided callable <function min at 0x0000016C7F1B5EA0> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  print(df_posiciones.groupby("Fo_Elemento")["Fecha"].agg([np.min, np.max]))
C:\Users\inaki.campo\AppData\Local\Temp\ipykernel_9160\3731279526.py:20: FutureWarning: The provided callable <function max at 0x0000016C7F1B5D80> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  print(df_posiciones.groupby("Fo_Elemento")["Fecha"].agg([np.min, np.max]))


Id_compouesta
20_2023-06-20 23:00:00    1
10_2022-01-01 00:00:00    1
10_2022-01-01 01:00:00    1
10_2022-01-01 02:00:00    1
10_2022-01-01 03:00:00    1
                         ..
10_2022-01-01 17:00:00    1
10_2022-01-01 16:00:00    1
10_2022-01-01 15:00:00    1
10_2022-01-01 14:00:00    1
10_2022-01-01 13:00:00    1
Name: count, Length: 121597, dtype: int64


In [ ]:
# # Borramos todas las filas de la tabla his_elemento_posiciones cuyo Fo_Elemento esté en la lista ids y cuya fecha sea anterior a 2027-01-01
# if not ids:
#     print("No hay Id en df_elementos.")
# else:
#     consulta_borrar = 'DELETE FROM public."HIS_Elemento_Posiciones" WHERE "Fo_Elemento" = ANY(%s::int[]) AND "Fecha" < %s;'
#     cursor.execute(consulta_borrar, (ids, '2027-01-01'))
#     conexion.commit()
#     print(f"Se han borrado las filas de HIS_Elemento_Posiciones para los Id {ids} con fecha anterior a 2027-01-01.")


Se han borrado las filas de HIS_Elemento_Posiciones para los Id [16, 18, 20, 23, 10, 11, 12] con fecha anterior a 2027-01-01.


## Cargamos datos 2022-2024 procesados y los insertamos en BBDD

In [8]:
# load C:\Users\inaki.campo\Desktop\Sacyr_Ionica\sw-modelo-rutas-trafico\data\datos_sacyr
df_2022 = pd.read_csv("./a3_22_24_filtered.csv")
# sustituimos el ultimo caracter del combre de la estacion por un 0
df_2022["Estación"] = df_2022["Estación"].str[:-1] + "0"
print(df_2022.head())

   Estación  Calzada         Dia      Hora  intensidad
0  CU-801-0        1  2022-01-01  00:00:00          54
1  CU-801-0        1  2022-01-01  01:00:00          95
2  CU-801-0        1  2022-01-01  02:00:00          68
3  CU-801-0        1  2022-01-01  03:00:00          39
4  CU-801-0        1  2022-01-01  04:00:00          25


In [9]:
# Modificamos el df_2022 para insertarlo en la tabla HIS_Elemento_Posiciones
# Recordemos: df_2022.columns = ["Estación", "Calzada", "Dia", "Hora", "intensidad"]

# Columnas:  Fo_Elemento	 Coordenadas	Ocupacion	IntensidadCarrilAsc	VelMedia	Fecha	Sentido_EsAscendente	Carriles	IntensidadCarrilDesc

# Fo_Elemento = 12  # M-108-0
# Coordenadas = POINT (40.39443 -3.63972)
# Ocupacion = Null
# IntensidadCarrilAsc = TO DO
# VelMedia = Null
# Fecha = TO DO
# Sentido_EsAscendente = Null ¿¿¿ Borrar ???
# Carriles = Null ¿¿¿ Borrar ???
# IntensidadCarrilDesc = TO DO

# Nos quedamos solo con las estaciones presentes en B_Elementos
df_tmp = df_2022[df_2022["Estación"].isin(df_elementos["Nombre"])].copy()

print(df_elementos["Nombre"])
display(df_tmp.head(5))

# --- INICIO DE LA CORRECCIÓN ---

# 1. Convertimos 'Dia' a datetime.
#    Si 'Dia' es solo la fecha (ej: "2022-01-01"), se pondrá a medianoche (00:00:00)
df_tmp["Dia"] = pd.to_datetime(df_tmp["Dia"], errors="coerce")

# 2. Procesamos la 'Hora'
#    Asumimos que 'Hora' es un string de tiempo (ej: "01:00" o "01:00:00")
#    Convertimos a datetime y extraemos el componente .dt.hour
hora_int = pd.to_datetime(df_tmp["Hora"], errors='coerce').dt.hour

#    Si el método anterior falla (ej: 'Hora' SÍ era un entero 0-23 pero
#    guardado como string "1", "2"), hora_int estará lleno de NaNs.
#    En ese caso, probamos tu método original (pd.to_numeric).
if hora_int.isnull().all():
    print("Formato 'HH:MM' fallido. Reintentando con 'pd.to_numeric'...")
    hora_int = pd.to_numeric(df_tmp["Hora"], errors='coerce')

# 3. Creamos la columna de hora final (rellenando posibles NaNs)
df_tmp["Hora_int"] = hora_int.fillna(0).astype(int)

# 4. Creamos la marca de tiempo completa (Fecha + Hora)
#    (He quitado el +5 años de prueba para mantener los datos reales)
df_tmp["Fecha"] = df_tmp["Dia"] + pd.to_timedelta(df_tmp["Hora_int"], unit="h")

# --- FIN DE LA CORRECCIÓN ---


# Ponemos los datos en formato wide donde cada fila es una estación+fecha
# Este pivot AHORA sí usará la 'Fecha' con granularidad horaria.
df_wide = df_tmp.pivot_table(
    index=["Estación", "Fecha"],
    columns="Calzada",
    values="intensidad",
    aggfunc="sum",
).reset_index()

print("tras wide (corregido)")
display(df_wide.head(5)) # Ahora deberías ver 24 filas por día (si hay datos)


# Mapeamos Fo_Elemento desde B_Elementos
id_map = df_elementos.set_index("Nombre")["Id"]
df_wide["Fo_Elemento"] = df_wide["Estación"].map(id_map)
# display(df_wide.head(5)) # Esta línea ya no es necesaria, se mostró antes

# Renombramos columnas
calzada_map = {1: "IntensidadCarrilAsc", 2: "IntensidadCarrilDesc"}
df_wide = df_wide.rename(columns={c: calzada_map[c] for c in calzada_map if c in df_wide.columns})

# Aseguramos que ambas columnas existan
for col in ("IntensidadCarrilAsc", "IntensidadCarrilDesc"):
    if col not in df_wide:
        df_wide[col] = np.nan
    df_wide[col] = pd.to_numeric(df_wide[col], errors="coerce")

# Limpieza y mapeo (tu código original está bien)
df_wide = df_wide.dropna(subset=["Fo_Elemento"])
df_wide["Fo_Elemento"] = df_wide["Fo_Elemento"].astype(int)

coord_from_pos = df_posiciones.groupby("Fo_Elemento")["Coordenadas"].first()
coord_from_elem = df_elementos.set_index("Id")["TR_Coordenadas"]
df_wide["Coordenadas"] = df_wide["Fo_Elemento"].map(coord_from_pos).fillna(
    df_wide["Fo_Elemento"].map(coord_from_elem)
)

df_wide["Ocupacion"] = None
df_wide["VelMedia"] = None
df_wide["Sentido_EsAscendente"] = None
df_wide["Carriles"] = pd.to_numeric(
    df_wide["Estación"].map(df_elementos.set_index("Nombre")["TR_Carriles"]), errors="coerce"
)

# Creamos el df final
df_his = (
    df_wide[
        [
            "Fo_Elemento",
            "Coordenadas",
            "Ocupacion",
            "IntensidadCarrilAsc",
            "VelMedia",
            "Fecha",
            "Sentido_EsAscendente",
            "Carriles",
            "IntensidadCarrilDesc",
        ]
    ]
    .sort_values(["Fo_Elemento", "Fecha"])
    .reset_index(drop=True)
)

display(df_his.head(5))
print(f"Filas preparadas: {len(df_his)}")

4      M-267-0
6      M-109-0
8      M-110-0
10    CU-801-0
15     M-108-0
16     M-139-0
17      M-43-0
Name: Nombre, dtype: object


,Estación,Calzada,Dia,Hora,intensidad
0,CU-801-0,1,2022-01-01,00:00:00,54
1,CU-801-0,1,2022-01-01,01:00:00,95
2,CU-801-0,1,2022-01-01,02:00:00,68
3,CU-801-0,1,2022-01-01,03:00:00,39
4,CU-801-0,1,2022-01-01,04:00:00,25


C:\Users\inaki.campo\AppData\Local\Temp\ipykernel_9160\2679985629.py:31: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  hora_int = pd.to_datetime(df_tmp["Hora"], errors='coerce').dt.hour


tras wide (corregido)


Calzada,Estación,Fecha,1,2
0,CU-801-0,2022-01-01 00:00:00,54.0,56.0
1,CU-801-0,2022-01-01 01:00:00,95.0,67.0
2,CU-801-0,2022-01-01 02:00:00,68.0,29.0
3,CU-801-0,2022-01-01 03:00:00,39.0,26.0
4,CU-801-0,2022-01-01 04:00:00,25.0,23.0


Calzada,Fo_Elemento,Coordenadas,Ocupacion,IntensidadCarrilAsc,VelMedia,Fecha,Sentido_EsAscendente,Carriles,IntensidadCarrilDesc
0,10,0101000020E610000059501894693244404CFF9254A618...,None,1967.0,None,2022-01-01 00:00:00,None,3.0,1074.0
1,10,0101000020E610000059501894693244404CFF9254A618...,None,1128.0,None,2022-01-01 01:00:00,None,3.0,1606.0
2,10,0101000020E610000059501894693244404CFF9254A618...,None,582.0,None,2022-01-01 02:00:00,None,3.0,916.0
3,10,0101000020E610000059501894693244404CFF9254A618...,None,302.0,None,2022-01-01 03:00:00,None,3.0,433.0
4,10,0101000020E610000059501894693244404CFF9254A618...,None,282.0,None,2022-01-01 04:00:00,None,3.0,274.0


Filas preparadas: 146533


## Insertamos datos de 2022 para las 7 espiras de las que tenemos predictivo

In [10]:
from psycopg2.extras import execute_values

## Insertamos los datos en la tabla HIS_Elemento_Posiciones
# Columnas destino en la tabla
cols_destino = (
    '"Fo_Elemento","Coordenadas","Ocupacion",'
    '"IntensidadCarrilAsc","VelMedia","Fecha","IntensidadCarrilDesc"'
)

# Preparar registros con conversión de NA -> None y tipos nativos
registros_insert = [
    (
        int(row.Fo_Elemento) if pd.notna(row.Fo_Elemento) else None,
        row.Coordenadas,
        row.Ocupacion,
        int(row.IntensidadCarrilAsc) if pd.notna(row.IntensidadCarrilAsc) else None,
        row.VelMedia,
        row.Fecha.to_pydatetime() if hasattr(row.Fecha, "to_pydatetime") else row.Fecha,
        int(row.IntensidadCarrilDesc) if pd.notna(row.IntensidadCarrilDesc) else None,
    )
    for _, row in df_his.iterrows()
]

sql_insert = f'INSERT INTO public."HIS_Elemento_Posiciones" ({cols_destino}) VALUES %s'

try:
    execute_values(cursor, sql_insert, registros_insert, page_size=1000)
    conexion.commit()
    print(f"Filas insertadas: {len(registros_insert)}")
except Exception as e:
    conexion.rollback()
    print(f"Error al insertar: {e}")

Filas insertadas: 146533


### Funciones de creación de input e inferencia de modelos predictivos. 24h y 3L


In [17]:
import holidays
import torch
import os
from omegaconf import OmegaConf
from models import get_model

def crear_input_predicciones_24h(nombre_espira, df_24h):
    print(f"----  Creacion input 24h para {nombre_espira}  -----")
    # fetures dia laborable: Calculamos las fechas del dia objetivo, anterior y posterior
    dia_objetivo = df_24h.iloc[-1]["Fecha"].date()
    print(f"Dia objetivo: {dia_objetivo}")
    dia_posterior = dia_objetivo + pd.Timedelta(days=1)
    print(f"Dia posterior: {dia_posterior}")
    dia_anterior = dia_objetivo - pd.Timedelta(days=1)
    print(f"Dia anterior: {dia_anterior}")

    # Creamos las ventanas de las dos calzadas, 1c es creciente y 2c es decreciente. Formato float32
    ventana_1c = df_24h[["IntensidadCarrilAsc"]].tail(24).T.to_numpy().astype(np.float32)
    ventana_2c = df_24h[["IntensidadCarrilDesc"]].tail(24).T.to_numpy().astype(np.float32)

    md_holidays = holidays.Spain(prov='MD')

    laborable = pd.to_datetime(dia_objetivo).dayofweek < 5
    # Comprobamos si el dia objetivo es laborable para mirar si fue festivo
    if laborable:
        laborable = not (pd.to_datetime(dia_objetivo) in md_holidays)
    new_col = np.array([int(laborable) for _ in range(24)]).reshape(1, 24)
    ventana_1c = np.concatenate((ventana_1c, new_col), axis=0)
    ventana_2c = np.concatenate((ventana_2c, new_col), axis=0)

    # Comprobamos si el dia anterior es laborable
    laborable_anterior = pd.to_datetime(dia_anterior).dayofweek < 5
    if laborable_anterior:
        laborable_anterior = not (pd.to_datetime(dia_anterior) in md_holidays)
    new_col = np.array([int(laborable_anterior) for _ in range(24)]).reshape(1, 24)
    ventana_1c = np.concatenate((ventana_1c, new_col), axis=0)
    ventana_2c = np.concatenate((ventana_2c, new_col), axis=0)

    # Comprobamos si el dia posterior es laborable
    laborable_posterior = pd.to_datetime(dia_posterior).dayofweek < 5
    if laborable_posterior:
        laborable_posterior = not (pd.to_datetime(dia_posterior) in md_holidays)
    new_col = np.array([int(laborable_posterior) for _ in range(24)]).reshape(1, 24)
    ventana_1c = np.concatenate((ventana_1c, new_col), axis=0)
    ventana_2c = np.concatenate((ventana_2c, new_col), axis=0)

    # Convertimos a tensores de torch
    ventana_1c = torch.tensor(ventana_1c).unsqueeze(0)  # shape (1, 4, 24)
    ventana_2c = torch.tensor(ventana_2c).unsqueeze(0)  # shape (1, 4, 24)

    # Hacemos reshape a (1, 24, 4)
    ventana_1c = ventana_1c.permute(0, 2, 1)  # shape (1, 24, 4)
    ventana_2c = ventana_2c.permute(0, 2, 1)  # shape (1, 24, 4)

    # convertimos a float32
    ventana_1c = ventana_1c.float()
    ventana_2c = ventana_2c.float()

    # load model from ./models/{id}.
    print(ventana_1c.shape, ventana_2c.shape)
    print("---- Fin creacion input ----")
    return ventana_1c, ventana_2c

def predicciones_24h(nombre_espira, input_c1, input_c2, device='cuda'):
    print(f"----  Predicciones 24h para {nombre_espira} -----")

    # predicciones 5-fold CV. Cargamos los 5 modelos de la id para la calzada 1
    preds_c1 = []
    preds_c2 = []
    for fold in range(5):
        # model path absoluto del fichero, cuya ubicacion desconocemos
        this_dir = os.getcwd()
        # this_dir = os.path.dirname(__file__)
        model_path_c1 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-1c", f"best_{fold}.pth")
        model_path_c2 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-2c", f"best_{fold}.pth")
        print(f"Model path C1: {model_path_c1}")
        print(f"Model path C2: {model_path_c2}")
        # load models config
        config_path_c1 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-1c", "config.yaml")
        config_path_c2 = os.path.join(this_dir, "models", f"{str(nombre_espira)}-2c", "config.yaml")
        config_c1 = OmegaConf.load(config_path_c1)
        config_c2 = OmegaConf.load(config_path_c2)
        print(config_c1)
        print(config_c2)
        # Instanciamos los objetos modelo
        input_size, output_size = input_c1[0].shape, 24
        print(f"Input size: {input_size}, Output size: {output_size}")
        model_1 = get_model(input_size, output_size, config_c1.model)
        model_2 = get_model(input_size, output_size, config_c2.model)
        model_1.to(device)
        model_2.to(device)
        input_c1 = input_c1.to(device)
        input_c2 = input_c2.to(device)
        print(f"Input C1 device: {input_c1.device}, Input C2 device: {input_c2.device}")
        # cargamos checkpint en el modelo
        load_checkpoint(model_1, model_path_c1, device)
        load_checkpoint(model_2, model_path_c2, device)
        model_1.eval()
        model_2.eval()
        # inferencia
        with torch.no_grad():
            pred_1 = model_1(input_c1).cpu().squeeze().numpy()  # shape (24,)
            pred_2 = model_2(input_c2).cpu().squeeze().numpy()  # shape (24,)
            preds_c1.append(pred_1)
            preds_c2.append(pred_2)
    # media de las predicciones
    pred_c1_mean = np.mean(preds_c1, axis=0)
    pred_c2_mean = np.mean(preds_c2, axis=0)
    print(f"Predicción sin desescalar calzada 1c (media 5 folds): {pred_c1_mean}")
    print(f"Predicción sin desescalar calzada 2c (media 5 folds): {pred_c2_mean}")
    print("-------- Fin predicciones ---------")
    return pred_c1_mean, pred_c2_mean


def load_checkpoint(model, checkpoint_path, device):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if "model_state" in checkpoint:
        state_dict = checkpoint["model_state"]
    else:
        state_dict = checkpoint

    model.load_state_dict(state_dict)


def compute_trasvase(row, trafico_minimo=800, flujo_maximo_carril_perjudicado=1600, factor_maximo_perjudicado_beneficiado=1.1):

    asc_is_bigger = row['IntensidadCarrilAsc'] >= row['IntensidadCarrilDesc']

    # identify the calzada with higher and lower flow
    if asc_is_bigger:
        high_flow = row['IntensidadCarrilAsc']
        low_flow = row['IntensidadCarrilDesc']
        n_high = row['n_carriles']
        n_low = row['n_carriles']
    else:
        high_flow = row['IntensidadCarrilDesc']
        low_flow = row['IntensidadCarrilAsc']
        n_high = row['n_carriles']
        n_low = row['n_carriles']

    # original average per lane using original number of carriles
    orig_avg_high = high_flow / n_high
    orig_avg_low = low_flow / n_low
    new_avg_high = high_flow / (n_high + 1)
    new_avg_low = low_flow / (n_low - 1)

    # if both flows are lower than 800 then no trasvase
    if orig_avg_high < trafico_minimo and orig_avg_low < trafico_minimo:
        if asc_is_bigger:
            return False, False, new_avg_high, new_avg_low
        else:
            return False, False, new_avg_low, new_avg_high
    
    # if the disadvantaged flow is higher than the maximum, dont recommend trasvase
    elif new_avg_low > flujo_maximo_carril_perjudicado:
        if asc_is_bigger:
            return False, False, new_avg_high, new_avg_low
        else:
            return False, False, new_avg_low, new_avg_high
    
    # if the modified flow of the disadvantaged lane is more than the advantaged lane, by factor_maximo_perjudicado_beneficiado, dont recommend trasvase
    elif new_avg_low > new_avg_high * factor_maximo_perjudicado_beneficiado:
        if asc_is_bigger:
            return False, False, new_avg_high, new_avg_low
        else:
            return False, False, new_avg_low, new_avg_high

    else:
        if asc_is_bigger:
            return True, False, new_avg_high, new_avg_low
        else:
            return False, True, new_avg_low, new_avg_high



### Carga ultimas 24h por cada espira, inferencia

In [19]:
# fecha_actual = pd.Timestamp.now()
fecha_actual = pd.to_datetime("2027-01-03 23:00:00")
print(f"Fecha actual: {fecha_actual}")

trafico_minimo=800
flujo_maximo_carril_perjudicado=1600
factor_maximo_perjudicado_beneficiado= 1.1
print(f"Tráfico mínimo: {trafico_minimo}, Factor de Flujo máximo de carril perjudicado respecto beneficiado: {factor_maximo_perjudicado_beneficiado}")

predicciones_espiras = {}
df_posiciones["Fecha"] = pd.to_datetime(df_posiciones["Fecha"], errors="coerce")
# seleccionamos las ultimas 48h registradas para cada Id de Fo_Elemento en df_posiciones
for Id, nombre_espira, n_carriles in df_elementos[["Id", "Nombre", "TR_Carriles"]].values:
    print(f"--- Creando Datos Input para espira {nombre_espira}  ID = {Id} ---")
    df_tmp = df_posiciones[df_posiciones["Fo_Elemento"] == Id].copy()
    # float
    df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]] = df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]].astype(np.float32)
    # Hacemos un escalado standard scaler de las columnas IntensidadCarrilAsc y IntensidadCarrilDesc
    medias = df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]].mean()
    stds = df_tmp[["IntensidadCarrilAsc", "IntensidadCarrilDesc"]].std()
    df_tmp["IntensidadCarrilAsc"] = (df_tmp["IntensidadCarrilAsc"] - medias["IntensidadCarrilAsc"]) / stds["IntensidadCarrilAsc"]
    df_tmp["IntensidadCarrilDesc"] = (df_tmp["IntensidadCarrilDesc"] - medias["IntensidadCarrilDesc"]) / stds["IntensidadCarrilDesc"]   
    print(f"Media y std de IntensidadCarrilAsc: {medias['IntensidadCarrilAsc']}, {stds['IntensidadCarrilAsc']}")
    print(f"Media y std de IntensidadCarrilDesc: {medias['IntensidadCarrilDesc']}, {stds['IntensidadCarrilDesc']}")

    if not df_tmp.empty:
        min_fecha = fecha_actual - pd.Timedelta(hours=48)
        df_48h = df_tmp[(df_tmp["Fecha"] > min_fecha) & (df_tmp["Fecha"] <= fecha_actual)]
        print(f"{nombre_espira}  Id: {Id}, Registros últimas 48h: {len(df_48h)}")

        # Por cada df, cogemos las ultimas 24 medidas (si hay tantas)
        if df_48h.shape[0] > 24:
            df_24h = df_48h.nlargest(24, "Fecha")
            df_24h = df_24h[["Fecha", "IntensidadCarrilAsc", "IntensidadCarrilDesc"]]
            print(df_24h.head(3))
            min_fecha_24h = df_24h["Fecha"].min()
            max_fecha_24h = df_24h["Fecha"].max()
            print(f"{nombre_espira}  Id: {Id}, Min Fecha 24h: {min_fecha_24h}, Max Fecha 24h: {max_fecha_24h}")
            print(f"Procedemos a predecir las próximas 24h para Id: {Id}")
            horas_a_predecir = pd.date_range(start=max_fecha_24h + pd.Timedelta(hours=1), periods=24, freq='h')
            print(f"horas a predecir: {horas_a_predecir[0]} - {horas_a_predecir[-1]}")

            input_c1, input_c2 = crear_input_predicciones_24h(nombre_espira, df_24h)

            pred_c1, pred_c2 = predicciones_24h(nombre_espira, input_c1, input_c2)
            
            # desescalamos las predicciones
            pred_c1 = (pred_c1 * stds["IntensidadCarrilAsc"]) + medias["IntensidadCarrilAsc"]
            pred_c2 = (pred_c2 * stds["IntensidadCarrilDesc"]) + medias["IntensidadCarrilDesc"]


            print(f"Predicción desescalada calzada 1c (media 5 folds): {pred_c1}")
            print(f"Predicción desescalada calzada 2c (media 5 folds): {pred_c2}")
            # Creamos un df con las predicciones y las horas a predecir
            df_preds = pd.DataFrame({
                "Fecha": horas_a_predecir,
                "IntensidadCarrilAsc": pred_c1,
                "IntensidadCarrilDesc": pred_c2,
                "n_carriles": n_carriles,
                "Fo_Elemento": Id
            })
            df_preds[['AbrirCarrilAsc', 'AbrirCarrilDesc', 'IntensidadPredCarrilAsc', 'IntensidadPredCarrilDesc']] = df_preds.apply(lambda row: compute_trasvase(row, trafico_minimo, flujo_maximo_carril_perjudicado, factor_maximo_perjudicado_beneficiado=1.1), axis=1,  result_type='expand')

            # Dividimos IntensidadCarrilAsc, IntensidadCarrilDesc por n_carriles para tener la intensidad por carril
            df_preds["IntensidadCarrilAsc"] = df_preds["IntensidadCarrilAsc"] / df_preds["n_carriles"]
            df_preds["IntensidadCarrilDesc"] = df_preds["IntensidadCarrilDesc"] / df_preds["n_carriles"]


            display(df_preds.head(3))

            # guardamos las predicciones en el diccionario con clave Id-1c y Id-2c
            predicciones_espiras[f"{Id}"] = df_preds

            print("--------------------------------------------------")

        else:
            print(f"Id: {Id} no tiene suficientes registros en las últimas 48h para predicciones (tiene {df_48h.shape[0]}).")
        
    else:
        print(f"Id: {Id} no tiene registros en df_posiciones.")


Fecha actual: 2027-01-03 23:00:00
Tráfico mínimo: 800, Factor de Flujo máximo de carril perjudicado respecto beneficiado: 1.1
--- Creando Datos Input para espira M-108-0  ID = 10 ---
Media y std de IntensidadCarrilAsc: 1848.4244384765625, 1143.395751953125
Media y std de IntensidadCarrilDesc: 2111.9599609375, 1370.5921630859375
M-108-0  Id: 10, Registros últimas 48h: 48
                 Fecha  IntensidadCarrilAsc  IntensidadCarrilDesc
71 2027-01-03 23:00:00            -1.205553             -1.086362
70 2027-01-03 22:00:00            -0.900322             -0.729582
69 2027-01-03 21:00:00            -0.240008             -0.386665
M-108-0  Id: 10, Min Fecha 24h: 2027-01-03 00:00:00, Max Fecha 24h: 2027-01-03 23:00:00
Procedemos a predecir las próximas 24h para Id: 10
horas a predecir: 2027-01-04 00:00:00 - 2027-01-04 23:00:00
----  Creacion input 24h para M-108-0  -----
Dia objetivo: 2027-01-03
Dia posterior: 2027-01-04
Dia anterior: 2027-01-02
torch.Size([1, 24, 4]) torch.Size([1, 24, 4

c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3607113742925706 and num_layers=1
  warnings.warn(
c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1529558592446795 and num_layers=1
  warnings.warn(


Input C1 device: cuda:0, Input C2 device: cuda:0
Model path C1: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-108-0-1c\best_1.pth
Model path C2: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-108-0-2c\best_1.pth
{'dataset': {'dataset_module': 'sacyr', 'root': './data/datasets', 'num_workers': 4, 'mode': 'reggresion', 'kfold': {'num_folds': 5, 'shuffle': True}, 'name': '108-24h-24h-3L-1c'}, 'scaler': {'name': 'standard'}, 'loss': {'name': 'mse'}, 'optimizer': {'name': 'adam', 'lr': 0.001, 'weight_decay': 0.0001, 'scheduler': 'step', 'step_size': 10, 'gamma': 0.1}, 'training': {'epochs': 100, 'batch_size': 1024, 'num_workers': 2, 'prefetch_factor': 3, 'seed': 42, 'device': 'cuda', 'log_dir': 'logs'}, 'scheduler': {'name': 'reduce_on_plateau', 'patience': 5, 'factor': 0.1}, 'metrics': [{'name': 'mae-sacyr-1h', 'prediction_horizon': 1}], 'callbacks': [{'name': 'checkpoint', 'dirpath': 'checkpoints/', 'monitor': 'Val_loss', 'mode': 'min'}, {'name': 'wandb_logger', '

,Fecha,IntensidadCarrilAsc,IntensidadCarrilDesc,n_carriles,Fo_Elemento,AbrirCarrilAsc,AbrirCarrilDesc,IntensidadPredCarrilAsc,IntensidadPredCarrilDesc
0,2027-01-04 00:00:00,245.689941,124.881144,3.0,10,False,False,184.267456,187.321716
1,2027-01-04 01:00:00,499.546631,413.939575,3.0,10,False,False,374.659973,620.909363
2,2027-01-04 02:00:00,579.508219,932.875651,3.0,10,False,False,869.262329,699.656738


--------------------------------------------------
--- Creando Datos Input para espira M-139-0  ID = 11 ---
Media y std de IntensidadCarrilAsc: 3504.16943359375, 2107.03955078125
Media y std de IntensidadCarrilDesc: 3236.4453125, 2001.6129150390625
M-139-0  Id: 11, Registros últimas 48h: 48
                   Fecha  IntensidadCarrilAsc  IntensidadCarrilDesc
8207 2027-01-03 23:00:00            -1.409166             -1.432567
8206 2027-01-03 22:00:00            -1.170918             -1.319159
8205 2027-01-03 21:00:00            -0.792662             -1.013405
M-139-0  Id: 11, Min Fecha 24h: 2027-01-03 00:00:00, Max Fecha 24h: 2027-01-03 23:00:00
Procedemos a predecir las próximas 24h para Id: 11
horas a predecir: 2027-01-04 00:00:00 - 2027-01-04 23:00:00
----  Creacion input 24h para M-139-0  -----
Dia objetivo: 2027-01-03
Dia posterior: 2027-01-04
Dia anterior: 2027-01-02
torch.Size([1, 24, 4]) torch.Size([1, 24, 4])
---- Fin creacion input ----
----  Predicciones 24h para M-139-0 -----

...

--------------------------------------------------
--- Creando Datos Input para espira M-43-0  ID = 12 ---
Media y std de IntensidadCarrilAsc: 1322.923095703125, 758.581787109375
Media y std de IntensidadCarrilDesc: 1748.6697998046875, 1070.0205078125
M-43-0  Id: 12, Registros últimas 48h: 48
                    Fecha  IntensidadCarrilAsc  IntensidadCarrilDesc
16895 2027-01-03 23:00:00            -1.138866             -1.461346
16894 2027-01-03 22:00:00            -0.454695             -1.317423
16893 2027-01-03 21:00:00            -0.055265             -1.051073
M-43-0  Id: 12, Min Fecha 24h: 2027-01-03 00:00:00, Max Fecha 24h: 2027-01-03 23:00:00
Procedemos a predecir las próximas 24h para Id: 12
horas a predecir: 2027-01-04 00:00:00 - 2027-01-04 23:00:00
----  Creacion input 24h para M-43-0  -----
Dia objetivo: 2027-01-03
Dia posterior: 2027-01-04
Dia anterior: 2027-01-02
torch.Size([1, 24, 4]) torch.Size([1, 24, 4])
---- Fin creacion input ----
----  Predicciones 24h para M-43-0 --

c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.14525356972625547 and num_layers=1
  warnings.warn(
c:\Users\inaki.campo\AppData\Local\miniconda3\envs\sacyr\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.22417876163845044 and num_layers=1
  warnings.warn(


Building CNN-LSTM model with config: {'cnn_filters': 153, 'dropout': 0.22417876163845044, 'hidden_size': 417, 'kernel_size': 2, 'name': 'cnn_lstm', 'num_layers': 1}
cnn_lstm module:  input_size: torch.Size([24, 4]), output_size: 24, config: {'cnn_filters': 153, 'dropout': 0.22417876163845044, 'hidden_size': 417, 'kernel_size': 2, 'name': 'cnn_lstm', 'num_layers': 1}
Input C1 device: cuda:0, Input C2 device: cuda:0
Model path C1: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-43-0-1c\best_2.pth
Model path C2: c:\Users\inaki.campo\Desktop\Sacyr_Ionica\service\models\M-43-0-2c\best_2.pth
{'dataset': {'dataset_module': 'sacyr', 'root': './data/datasets', 'num_workers': 4, 'mode': 'reggresion', 'kfold': {'num_folds': 5, 'shuffle': True}, 'name': '43-24h-24h-3L-1c'}, 'scaler': {'name': 'standard'}, 'loss': {'name': 'mse'}, 'optimizer': {'name': 'adam', 'lr': 0.001, 'weight_decay': 0.0001, 'scheduler': 'step', 'step_size': 10, 'gamma': 0.1}, 'training': {'epochs': 100, 'batch_size

,Fecha,IntensidadCarrilAsc,IntensidadCarrilDesc,n_carriles,Fo_Elemento,AbrirCarrilAsc,AbrirCarrilDesc,IntensidadPredCarrilAsc,IntensidadPredCarrilDesc
0,2027-01-04 00:00:00,216.344666,38.798869,3.0,12,False,False,162.258499,58.198303
1,2027-01-04 01:00:00,417.873047,94.101278,3.0,12,False,False,313.404785,141.151917
2,2027-01-04 02:00:00,575.873698,252.993510,3.0,12,False,False,431.905273,379.490265


--------------------------------------------------


### Insertamos las predicciones en la tabla 

In [20]:
conexion = psycopg2.connect(
    host="srvbu-bd",
    port="5432",
    database="BITAL_SACYR_ICONICA",
    user="adminpg",
    password="KRZEy5fxQyNmmVjeUseQ",
)

cursor = conexion.cursor()

for k, v in predicciones_espiras.items():
    # insert into B_PrediccionTraficoe
    # Preparamos e insertamos (omitimos n_carriles)
    cols_sql = '"Fo_Elemento","Fecha","IntensidadCarrilAsc","IntensidadCarrilDesc","AbrirCarrilAsc","AbrirCarrilDesc","IntensidadPredCarrilAsc","IntensidadPredCarrilDesc"'
    registros = [
        (
            int(r.Fo_Elemento),
            r.Fecha.to_pydatetime(),
            float(r.IntensidadCarrilAsc),
            float(r.IntensidadCarrilDesc),
            bool(r.AbrirCarrilAsc),
            bool(r.AbrirCarrilDesc),
            float(r.IntensidadPredCarrilAsc),
            float(r.IntensidadPredCarrilDesc),
        )
        for r in v.itertuples(index=False)
    ]
    if registros:
        sql = f'INSERT INTO public."B_PrediccionTrafico" ({cols_sql}) VALUES %s'
        try:
            execute_values(cursor, sql, registros, page_size=100)
            conexion.commit()
            print(f"Insertadas {len(registros)} filas para Fo_Elemento {k}")
        except Exception as e:
            conexion.rollback()
            print(f"Error insertando Fo_Elemento {k}: {e}")
    

Insertadas 24 filas para Fo_Elemento 10
Insertadas 24 filas para Fo_Elemento 11
Insertadas 24 filas para Fo_Elemento 12
